In [2]:
import pandas as pd
import random
import os

In [33]:
def generate_practice_lists(csv_path, output_js_path, set_number):
    # Read the CSV (no header)
    df = pd.read_csv(csv_path, header=None)

    # Prepare study list from first column (old images)
    study_list = []
    for i, img in enumerate(df[0].dropna(), start=1):
        study_list.append({
            'block': 0,
            'phase': 'study',
            'trial': i,
            'stimulus': f'stimulus/{img}',
            'stim_type': 'old',
            'correct_response': 'f'
        })

    # Prepare test list from old and new images
    test_list = []
    for img in df[0].dropna():
        test_list.append({
            'block': 0,
            'phase': 'test',
            'stimulus': f'stimulus/{img}',
            'stim_type': 'old',
            'correct_response': 'f'
        })
    for img in df[1].dropna():
        test_list.append({
            'block': 0,
            'phase': 'test',
            'stimulus': f'stimulus/{img}',
            'stim_type': 'new',
            'correct_response': 'j'
        })

    # Shuffle test list and assign trial numbers
    random.shuffle(test_list)
    for i, item in enumerate(test_list, start=1):
        item['trial'] = i

    # Combine all unique image paths for preloading
    all_images = set(f'stimulus/{img}' for img in pd.concat([df[0], df[1]]).dropna())

    # Write to JavaScript file
    with open(output_js_path, 'w') as f:
        # Write study list
        f.write(f'var practice_study_list = [\n')
        for item in study_list:
            f.write(f'    {item},\n')
        f.write('];\n\n')

        # Write test list
        f.write(f'var practice_test_list = [\n')
        for item in test_list:
            f.write(f'    {item},\n')
        f.write('];\n\n')

        # Write practice_images list
        f.write(f'var practice_images = [\n')
        for img in sorted(all_images):
            f.write(f'    "{img}",\n')
        f.write('];\n')

# === Generate for all 48 files ===
for i in range(1, 49):
    csv_file = f'stim_csv/pracset_{i}.csv'
    js_file = f'stim_lists/practice_{i}.js'
    if os.path.exists(csv_file):
        generate_practice_lists(csv_file, js_file, i)
    else:
        print(f"Missing file: {csv_file}")


In [34]:
def convert_all_csvs_to_js(csv_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    for file in os.listdir(csv_dir):
        if (file.endswith(".csv")) and (file.startswith("stimset")):
            file_path = os.path.join(csv_dir, file)
            list_id = os.path.splitext(file)[0]  # e.g., "stimset_1"
            output_file = os.path.join(output_dir, f"{list_id}.js")

            try:
                df = pd.read_csv(file_path, header=None, names=["old", "new"])

                old_js = f"var old_images = [\n" + ",\n".join(
                    f"  'stimulus/{img}'" for img in df['old'].dropna()
                ) + "\n];\n"

                new_js = f"var new_images = [\n" + ",\n".join(
                    f"  'stimulus/{img}'" for img in df['new'].dropna()
                ) + "\n];\n"

                with open(output_file, 'w') as f:
                    f.write(old_js + "\n" + new_js)

                print(f"✅ Converted {file} → {output_file}")

            except Exception as e:
                print(f"❌ Error processing {file}: {e}")

# Example usage
csv_input_dir = "stim_csv"
js_output_dir = "stim_lists"

convert_all_csvs_to_js(csv_input_dir, js_output_dir)


✅ Converted stimset_47.csv → stim_lists/stimset_47.js
✅ Converted stimset_46.csv → stim_lists/stimset_46.js
✅ Converted stimset_44.csv → stim_lists/stimset_44.js
✅ Converted stimset_45.csv → stim_lists/stimset_45.js
✅ Converted stimset_41.csv → stim_lists/stimset_41.js
✅ Converted stimset_40.csv → stim_lists/stimset_40.js
✅ Converted stimset_42.csv → stim_lists/stimset_42.js
✅ Converted stimset_43.csv → stim_lists/stimset_43.js
✅ Converted stimset_30.csv → stim_lists/stimset_30.js
✅ Converted stimset_24.csv → stim_lists/stimset_24.js
✅ Converted stimset_1.csv → stim_lists/stimset_1.js
✅ Converted stimset_18.csv → stim_lists/stimset_18.js
✅ Converted stimset_19.csv → stim_lists/stimset_19.js
✅ Converted stimset_25.csv → stim_lists/stimset_25.js
✅ Converted stimset_31.csv → stim_lists/stimset_31.js
✅ Converted stimset_27.csv → stim_lists/stimset_27.js
✅ Converted stimset_33.csv → stim_lists/stimset_33.js
✅ Converted stimset_2.csv → stim_lists/stimset_2.js
✅ Converted stimset_3.csv → stim

In [28]:
csv_input_dir = "stim_csv"

for file in os.listdir(csv_input_dir):
    if (file.endswith(".csv")) and (file.startswith("stimset")):
        file_path = os.path.join(csv_input_dir, file)
        list_id = file.split(".")[0].split("_")[-1]

        stim_df = pd.read_csv(file_path, header=None)
        prac_path = os.path.join(csv_input_dir, f"pracset_{list_id}.csv")
        prac_df = pd.read_csv(prac_path, header=None)

        # check old new non-repeat for practice list
        prac_overlap = set(prac_df[0]) & set(prac_df[1])
        if len(prac_overlap) > 0:
            print(f"{prac_path} duplicate images in practice list")
            continue

        stim_overlap = set(stim_df[0]) & set(stim_df[1])
        if len(stim_overlap) > 0:
            print(f"{file_path} duplicate images in stim list")
            continue

        stim_prac_overlap = set(prac_df.values.flatten()) & set(stim_df.values.flatten())
        if len(stim_prac_overlap) > 0:
            print(f"{list_id} list has overlap btw practice and stim lists")
            continue